# Part 1: BST Traversa

## Problem 1

In [152]:
from enum import Enum
from TreeTraversal import BinaryTree, DFSTraversal

class DFSTraversalTypes(Enum):
    PREORDER = 1
    INORDER = 2
    POSTORDER = 3
    
input_array = [3, 9, 2, 11]

bt = BinaryTree() # initialize tree
for val in input_array: # fill it in
    bt.insert(val)


# step through tree in given order and print results
traversal = DFSTraversal(bt, DFSTraversalTypes.INORDER) 
for val in traversal: 
    print(val)

# output
# 2
# 3
# 9
# 11

2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2

2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2

11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
1

9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9

3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3

2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2

11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
1

9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9
11
2
3
9

KeyboardInterrupt: 